# Prolific study evaluation
This notebook loads and processes the raw data collected on Supabase from the online human study done via Prolific.
It reconstructs participant-level and trial-level responses from the stored jsPsych logs,
and prepares clean analysis tables for downstream statistical evaluation.

In [3]:
import os
from dotenv import load_dotenv
from supabase import create_client
import pandas as pd
from pathlib import Path

load_dotenv()

SUPABASE_URL = "https://utwhgfveotpusdjopcnl.supabase.co"
SUPABASE_KEY = os.environ["SUPABASE_SERVICE_ROLE"]

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

In [8]:
# Read out results table and print as dataframe
response = supabase.table("results").select("*").execute()

rows = response.data
df_raw = pd.DataFrame(rows)

display(df_raw.head())
print(df_raw.shape)

,id,prolific_pid,profile_id,exit_reason,created_at,data,exit_time,experiment_start_time,profile_index
0,78f2fd02-b50f-4c85-83f4-d06096a2a8c0,DEBUG,debug_profile,completed,2025-12-30T18:48:19.397257+00:00,"[{'rt': 1414, 'exit_time': 1767120498908, 'pro...",1767120498908,1767120475173,-1
1,6a52faf5-16be-40ed-9473-6ea0e02bf53f,DEBUG,debug_profile,completed,2025-12-30T19:30:51.544457+00:00,"[{'rt': 1409, 'exit_time': 1767123050529, 'pro...",1767123050529,1767123026955,-1
2,59ff4bcd-653b-4682-8396-580955516b7e,TEST_1d0e4153-c4e9-44c0-8ef2-7e18a66ae979,profile_25_first,completed,2025-12-30T19:36:50.817149+00:00,"[{'rt': 1125, 'exit_time': 1767123407781, 'tim...",1767123407781,1767123156882,34
3,470d91eb-0351-4747-b694-518034c2a015,DEBUG,debug_profile,completed,2025-12-30T20:02:00.778582+00:00,"[{'rt': 822, 'exit_time': 1767124919569, 'time...",1767124919569,1767124884483,-1
4,36e6c355-79cd-4fa3-b731-ba53a25e9bf0,TEST_4094a6c0-f96f-498d-9e05-7b4f1c0ef206,profile_32_last,failed_distractor,2025-12-30T20:25:33.516265+00:00,"[{'rt': 1842, 'exit_time': 1767126332543, 'tim...",1767126332543,1767126207820,51


(49, 9)


In [21]:
rows = []

for _, row in df_raw.iterrows():
    pid = row.get("prolific_pid")
    profile_id = row.get("profile_id")
    exit_reason = row.get("exit_reason")

    for trial in row["data"]:
        flat = {
            "PROLIFIC_PID": pid,
            "profile_id": profile_id,
            "exit_reason": exit_reason,
        }
        flat.update(trial)
        rows.append(flat)

df = pd.DataFrame(rows)
display(df.head(10))
print(df.shape)

,PROLIFIC_PID,profile_id,exit_reason,rt,exit_time,subject_id,trial_type,trial_index,time_elapsed,view_history,...,sanity_id,question_order,response_index,correct_response,slider_start,timed_out,timeout_time,image_path,distractor_errors,certainty
0,DEBUG,debug_profile,completed,1414.0,1767120498908,DEBUG,instructions,0,1439,"[{'page_index': 0, 'viewing_time': 1413}]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DEBUG,debug_profile,completed,1437.0,1767120498908,DEBUG,html-button-response,1,2883,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DEBUG,debug_profile,completed,1765.0,1767120498908,DEBUG,html-button-response,2,4659,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DEBUG,debug_profile,completed,3127.0,1767120498908,DEBUG,survey-likert,3,7788,NaN,...,1.0,[0],4.0,Strongly Disagree,NaN,NaN,NaN,NaN,NaN,NaN
4,DEBUG,debug_profile,completed,1456.0,1767120498908,DEBUG,html-button-response,4,9251,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,DEBUG,debug_profile,completed,4412.0,1767120498908,DEBUG,survey-text,5,13692,NaN,...,2.0,NaN,NaN,BLUE,NaN,NaN,NaN,NaN,NaN,NaN
6,DEBUG,debug_profile,completed,1464.0,1767120498908,DEBUG,html-button-response,6,15161,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,DEBUG,debug_profile,completed,1632.0,1767120498908,DEBUG,survey-likert,7,16797,NaN,...,3.0,[0],2.0,Option 3,NaN,NaN,NaN,NaN,NaN,NaN
8,DEBUG,debug_profile,completed,1551.0,1767120498908,DEBUG,html-button-response,8,18352,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,DEBUG,debug_profile,completed,2826.0,1767120498908,DEBUG,survey-likert,9,21181,NaN,...,4.0,[0],0.0,Strongly Disagree,NaN,NaN,NaN,NaN,NaN,NaN


(2815, 34)


In [13]:
print("Task type counts:")
print(df["task_type"].value_counts())
print("\nExit reason counts:")
print(df_raw["exit_reason"].value_counts(dropna=False))

Task type counts:
task_type
color_judgment    2538
sanity             150
introspection       48
warning             30
Name: count, dtype: int64

Exit reason counts:
exit_reason
completed            24
failed_distractor    20
failed_attention      5
Name: count, dtype: int64


In [23]:
# Extract only valid color judgments for analysis)
df_color = df[
    (df["task_type"] == "color_judgment") &
    (df["exit_reason"] == "completed")
].copy()

len(df_color)

2115

In [22]:
trials_per_pid = df.groupby("PROLIFIC_PID").size()

trials_per_pid.describe()


count     24.000000
mean     117.291667
std       46.275620
min       36.000000
25%      107.000000
50%      107.000000
75%      108.000000
max      258.000000
dtype: float64